In [1]:
import os
import requests
import getpass
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

In [2]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("API key for Langsmith: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("API key for OpenAI: ")

In [3]:
class Website:
    """"
    Utility class to represent a website that we have scraped.
    """
    def __init__(self, url):
        """
        Create a Website object from the give url using BeautifulSoup.
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
website = Website("https://en.wikipedia.org/wiki/Large_language_model")
print(website.title)
print()
print(website.text)

Large language model - Wikipedia

Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Special pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
History
2
Dataset preprocessing
Toggle Dataset preprocessing subsection
2.1
Tokenization
2.1.1
BPE
2.1.2
Problems
2.2
Dataset cleaning
2.3
Synthetic data
3
Training and architecture
Toggle Training and architecture subsection
3.1
Reinforcement learning from human feedback (RLHF)
3.2
Instruction tuning
3.3
Mixture of experts
3.4
Prompt engineering, attention mechanism, and context window
3.5
Infrastructure
4
Training cost
5
Tool use
6
Agency
7
Compression
8
Multimodality
9
Reasoning
10
Properties
Toggle Properties subsection
10.1
Scal

In [5]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [7]:
print(user_prompt_for(website))

You are looking at a website titled Large language model - Wikipedia
The contents of this website is as follows; provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Special pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
History
2
Dataset preprocessing
Toggle Dataset preprocessing subsection
2.1
Tokenization
2.1.1
BPE
2.1.2
Problems
2.2
Dataset cleaning
2.3
Synthetic data
3
Training and architecture
Toggle Training and architecture subsection
3.1
Reinforcement learning from human feedback (RLHF)
3.2
Instruction tuning
3.3
Mixture of experts
3.4
Prompt engin

In [8]:
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [9]:
template = ChatPromptTemplate([
    ("system", system_prompt),
    ("human", "{user_prompt}")
])

prompt = template.invoke({"user_prompt": user_prompt_for(website)})

response = model.invoke(prompt)
print(response.content)

# Summary of "Large Language Model - Wikipedia"

The Wikipedia entry for "Large Language Model" describes a type of machine learning model used for natural language processing (NLP). These models, particularly the generative pretrained transformers (GPTs), are trained on extensive text datasets using self-supervised learning techniques. Key aspects covered include:

## Key Sections

- **History**: Traces the evolution of large language models from earlier statistical models to the introduction of neural networks, notably the transformer architecture in 2017. Important models like BERT and the GPT series are highlighted for their impact on the field.

- **Dataset Preprocessing**: Discusses methods such as tokenization, dataset cleaning, and the use of synthetic data in training.

- **Training and Architecture**: Explores techniques like Reinforcement Learning from Human Feedback (RLHF), instruction tuning, and the mixture of experts approach, emphasizing the complexity and resource dema

In [10]:
def summarize(url):
    website = Website(url)
    prompt = template.invoke({"user_prompt": user_prompt_for(website)})
    response = init_chat_model("gpt-4o-mini", model_provider="openai").invoke(prompt)
    return response.content

In [11]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [12]:
display_summary("https://en.wikipedia.org/wiki/Large_language_model")

# Summary of the "Large Language Model" Wikipedia Page

The Wikipedia page on Large Language Models (LLMs) provides an extensive overview of this class of machine learning models specifically designed for natural language processing (NLP) tasks. LLMs are characterized by their capacity to generate human-like text, relying heavily on a vast amount of training data processed via self-supervised learning methods. The most advanced LLMs are typically based on transformer architectures.

## Key Sections Covered:

- **History**: The section discusses the evolution of LLMs from early statistical models in the 1990s to the advent of neural networks and the transformer architecture in 2017. It highlights milestones like BERT and the progression to models like GPT-2 and GPT-3, culminating with the introduction of GPT-4 and its multimodal capabilities by OpenAI.

- **Dataset Preprocessing**: This part focuses on the steps involved in preparing datasets for training, including tokenization (with methods like Byte Pair Encoding), cleaning to remove low-quality or toxic data, and the use of synthetic data when necessary.

- **Training and Architecture**: This section details various techniques used for training LLMs, including reinforcement learning from human feedback (RLHF), instruction tuning, and the mixture of experts approach. It also discusses the complexity of infrastructure required for training large models.

- **Properties and Capabilities**: It explores the emergent abilities that larger models exhibit, such as in-context learning and scaling laws that predict performance based on parameters and dataset size.

- **Evaluation**: The evaluation methods for LLMs are reviewed, focusing on perplexity, benchmarks, and the limitations of existing metrics.

- **Wider Impact**: This segment outlines the potential societal implications of LLMs, including concerns over copyright, misinformation, algorithmic bias, and their environmental impact due to energy demands.

## Recent Developments:
- A notable announcement in late 2024 discussed the emergence of reasoning models, specifically mentioning OpenAI's o1 model that shows improvements in structured problem-solving compared to traditional LLMs.

Overall, the Wikipedia page serves as a comprehensive resource for understanding large language models, their historical development, underlying technologies, evaluative metrics, and their societal implications.